In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType, DateType


In [ ]:
# Session
spark = SparkSession.builder \
    .appName("StreamingPreciosCombustible") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


25/05/31 11:40:43 WARN Utils: Your hostname, MacBook-Pro-de-Nick.local resolves to a loopback address: 127.0.0.1; using 192.168.1.131 instead (on interface en0)
25/05/31 11:40:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/31 11:40:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Estructura
schema = StructType() \
    .add("id_precio", IntegerType()) \
    .add("id_estacion", IntegerType()) \
    .add("id_combustible", IntegerType()) \
    .add("fecha_toma_datos", DateType()) \
    .add("precio", FloatType())

In [ ]:
# Carga Stream
df_stream = spark.readStream \
    .option("sep", ",") \
    .schema(schema) \
    .csv("/Users/nickelao/spark/input/")

In [ ]:
# Quito nulls
df_stream = df_stream.filter("id_combustible IS NOT NULL")

In [ ]:
# carga combustibles y rename
combustibles = spark.read.option("header", True).csv("/Users/nickelao/spark/combustibles_actualizado.csv")
combustibles = combustibles.withColumnRenamed("ID_Combustible", "id_combustible")


In [ ]:
# precio avg
precios_agrupados = df_stream.groupBy("id_combustible").agg(avg("precio").alias("precio_medio"))

In [ ]:
# join para darle nombre a los id de combustible
precios_con_nombre = precios_agrupados.join(combustibles, on="id_combustible", how="left")

In [ ]:
# Query sin el join
query = precios_agrupados.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+------------------+
|id_combustible|precio_medio      |
+--------------+------------------+
|12            |0.9369795695427925|
|1             |1.506478770663414 |
|13            |1.351838947942593 |
|16            |899.7777777777778 |
|6             |1.4178542954538722|
|3             |1.6193705243244385|
|5             |1.5993077021378737|
|15            |10.0              |
|9             |1.1953636407852173|
|4             |1.6714145589114964|
|8             |19.933939746282633|
|7             |1.5212354476781693|
|10            |2.106666684150696 |
|11            |1.405466663837433 |
|14            |1.2453499948978424|
|2             |1.5806785694190435|
+--------------+------------------+



In [ ]:
# Query con el join
query = precios_con_nombre.select("Nombre", "precio_medio") \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()

NameError: name 'precios_con_nombre' is not defined

In [ ]:
query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+----------------------------------+------------------+
|Nombre                            |precio_medio      |
+----------------------------------+------------------+
|Precio gases licuados del petróleo|0.9369795695427925|
|NULL                              |NULL              |
|Precio gasolina 95 E5             |1.506478770663414 |
|Precio gas natural comprimido     |1.351838947942593 |
|Precio gasóleo de uso marítimo    |899.7777777777778 |
|Precio gasóleo A                  |1.4178542954538722|
|Precio gasolina 95 E5 Premium     |1.6193705243244385|
|Precio gasolina 98 E10            |1.5993077021378737|
|Precio hidrógeno                  |10.0              |
|Precio gasóleo C                  |1.1953636407852173|
|Precio gasolina 98 E5             |1.6714145589114964|
|Precio gasóleo B                  |19.933939746282633|
|Precio gasóleo Premium            |1.5212354476781693|
|Precio

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/nickelao/spark/venv/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/nickelao/spark/venv/lib/python3.13/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
KeyboardInterrupt


KeyboardInterrupt: 